**About Book Crossing Dataset**<br>

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed by 0.

Reference: http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

**Objective**

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering approaches.

In [48]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

#### Execute the below cell to load the datasets

In [3]:
#Loading data
import pandas as pd
books = pd.read_csv("books/books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']



b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:27

### Check no.of records and features given in each dataset

In [4]:
books.shape

(271360, 8)

In [5]:
#I have put users and ratings somwhere below where it is used, to avoid running dataset load again and again

## Exploring books dataset

In [6]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Drop last three columns containing image URLs which will not be required for analysis

In [7]:
columns = ['imageUrlS', 'imageUrlM', 'imageUrlL']
books.drop(columns, axis=1, inplace=True)

In [8]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


**yearOfPublication**

### Check unique values of yearOfPublication


In [9]:
#to eye ball the data I am exporting it into csv
books.to_csv('books_pruned.csv')

In [10]:
books['yearOfPublication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

As it can be seen from above that there are some incorrect entries in this field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as yearOfPublication in dataset due to some errors in csv file.


Also some of the entries are strings and same years have been entered as numbers in some places. We will try to fix these things in the coming questions.

### Check the rows having 'DK Publishing Inc' as yearOfPublication

In [11]:
books.loc[books['yearOfPublication'] == "DK Publishing Inc",'yearOfPublication']


209538    DK Publishing Inc
221678    DK Publishing Inc
Name: yearOfPublication, dtype: object

### Drop the rows having `'DK Publishing Inc'` and `'Gallimard'` as `yearOfPublication`

In [12]:
#books.drop(books.loc[books['yearOfPublication'] == "DK Publishing Inc" or books['yearOfPublication']=="Gallimard"], axis = 0)
junk_yop = ['DK Publishing Inc','Gallimard']
for i in junk_yop:
    books.drop(books.loc[books['yearOfPublication'] == i].index, axis=0, inplace=True)
    print(books.shape)

(271358, 5)
(271357, 5)


### Change the datatype of yearOfPublication to 'int'

In [13]:
books['yearOfPublication']=books['yearOfPublication'].astype(int)
books.dtypes

ISBN                 object
bookTitle            object
bookAuthor           object
yearOfPublication     int64
publisher            object
dtype: object

### Drop NaNs in `'publisher'` column


In [14]:
books.info()
#shaows that we have 2 null columns in publisher

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271357 entries, 0 to 271359
Data columns (total 5 columns):
ISBN                 271357 non-null object
bookTitle            271357 non-null object
bookAuthor           271356 non-null object
yearOfPublication    271357 non-null int64
publisher            271355 non-null object
dtypes: int64(1), object(4)
memory usage: 12.4+ MB


In [15]:
books.publisher.isna().value_counts()

False    271355
True          2
Name: publisher, dtype: int64

In [16]:
books['publisher'].dropna(inplace=True)
books.publisher.isna().value_counts()

False    271355
Name: publisher, dtype: int64

## Exploring Users dataset

In [17]:
users = pd.read_csv('books/users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

In [18]:
print(users.shape)
users.head(100)

(278858, 3)


,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
5,6,"santa monica, california, usa",61.0
6,7,"washington, dc, usa",NaN
7,8,"timmins, ontario, canada",NaN
8,9,"germantown, tennessee, usa",NaN
9,10,"albacete, wisconsin, spain",26.0


### Get all unique values in ascending order for column `Age`

In [19]:
users['Age'].unique()

array([ nan,  18.,  17.,  61.,  26.,  14.,  25.,  19.,  46.,  55.,  32.,
        24.,  20.,  34.,  23.,  51.,  31.,  21.,  44.,  30.,  57.,  43.,
        37.,  41.,  54.,  42.,  50.,  39.,  53.,  47.,  36.,  28.,  35.,
        13.,  58.,  49.,  38.,  45.,  62.,  63.,  27.,  33.,  29.,  66.,
        40.,  15.,  60.,   0.,  79.,  22.,  16.,  65.,  59.,  48.,  72.,
        56.,  67.,   1.,  80.,  52.,  69.,  71.,  73.,  78.,   9.,  64.,
       103., 104.,  12.,  74.,  75., 231.,   3.,  76.,  83.,  68., 119.,
        11.,  77.,   2.,  70.,  93.,   8.,   7.,   4.,  81., 114., 230.,
       239.,  10.,   5., 148., 151.,   6., 101., 201.,  96.,  84.,  82.,
        90., 123., 244., 133.,  91., 128.,  94.,  85., 141., 110.,  97.,
       219.,  86., 124.,  92., 175., 172., 209., 212., 237.,  87., 162.,
       100., 156., 136.,  95.,  89., 106.,  99., 108., 210.,  88., 199.,
       147., 168., 132., 159., 186., 152., 102., 116., 200., 115., 226.,
       137., 207., 229., 138., 109., 105., 228., 18

Age column has some invalid entries like nan, 0 and very high values like 100 and above

### Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

In [20]:
users.loc[(users['Age']<5) | (users['Age']>90),'Age']= np.nan
users['Age'].isna().value_counts()

False    166784
True     112074
Name: Age, dtype: int64

### Replace null values in column `Age` with mean

In [21]:
users['Age'] = users['Age'].replace(np.nan,users['Age'].mean())
users['Age'].isna().value_counts()

False    278858
Name: Age, dtype: int64

### Change the datatype of `Age` to `int`

In [22]:
users['Age'] = users['Age'].astype(int)

In [23]:
print(sorted(users.Age.unique()))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


## Exploring the Ratings Dataset

In [24]:
ratings = pd.read_csv('books/ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

### check the shape

In [25]:
ratings.shape

(1149780, 3)

In [26]:
n_users = users.shape[0]
n_books = books.shape[0]

In [27]:
ratings.head(5)

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

In [28]:
#ISBN is a unique book identifier, so we can use it as a key to join books and ratings table
#we can then remove the rows that contain Nan from user
books_ratings = ratings.join(books.set_index('ISBN'), on ="ISBN")


In [29]:
books_ratings.head(20)

,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002.0,Ballantine Books
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001.0,Heinle
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996.0,Warner Books
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999.0,Cambridge University Press
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001.0,Cambridge University Press
5,276733,2080674722,0,Les Particules Elementaires,Michel Houellebecq,1998.0,Flammarion
6,276736,3257224281,8,NaN,NaN,NaN,NaN
7,276737,0600570967,6,NaN,NaN,NaN,NaN
8,276744,038550120X,7,A Painted House,JOHN GRISHAM,2001.0,Doubleday
9,276745,342310538,10,NaN,NaN,NaN,NaN


In [30]:
books_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 7 columns):
userID               1149780 non-null int64
ISBN                 1149780 non-null object
bookRating           1149780 non-null int64
bookTitle            1031132 non-null object
bookAuthor           1031131 non-null object
yearOfPublication    1031132 non-null float64
publisher            1031130 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 61.4+ MB


In [31]:
#removing the extra columns that got added due to join
books_ratings.dropna(inplace=True)
columns = ['bookTitle','bookAuthor', 'yearOfPublication','publisher']
ratings = books_ratings.drop(columns, axis=1)
ratings.head(10)
ratings.shape

(1031129, 3)

### Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

In [32]:
#same process for users and ratings as well. Here the user's data set has unique userID but ratings dataset has 

users_ratings = ratings.join(users.set_index('userID'), on ='userID')
users_ratings.head(100)

,userID,ISBN,bookRating,Location,Age
0,276725,034545104X,0,"tyler, texas, usa",34
1,276726,0155061224,5,"seattle, washington, usa",34
2,276727,0446520802,0,"h, new south wales, australia",16
3,276729,052165615X,3,"rijeka, n/a, croatia",16
4,276729,0521795028,6,"rijeka, n/a, croatia",16
5,276733,2080674722,0,"paris, n/a, france",37
8,276744,038550120X,7,"torrance, california, usa",34
10,276746,0425115801,0,"fort worth, ,",34
11,276746,0449006522,0,"fort worth, ,",34
12,276746,0553561618,0,"fort worth, ,",34


In [33]:
users_ratings.shape
users_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031129 entries, 0 to 1149778
Data columns (total 5 columns):
userID        1031129 non-null int64
ISBN          1031129 non-null object
bookRating    1031129 non-null int64
Location      1031129 non-null object
Age           1031129 non-null int64
dtypes: int64(3), object(2)
memory usage: 47.2+ MB


In [34]:
users_ratings.dropna(inplace=True)
columns = ['Location','Age']
ratings = users_ratings.drop(columns, axis=1)
ratings.head(10)
ratings.shape

(1031129, 3)

In [35]:
ratings.head(10)

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0
8,276744,038550120X,7
10,276746,0425115801,0
11,276746,0449006522,0
12,276746,0553561618,0


### Consider only ratings from 1-10 and leave 0s in column `bookRating`

In [36]:
ratings.drop(ratings.loc[ratings['bookRating'] <= 0].index, axis=0, inplace=True)
ratings.shape

(383838, 3)

### Find out which rating has been given highest number of times

In [37]:
ratings['bookRating'].mode()
#Rating 8 is the highest given rating

0    8
dtype: int64

### **Collaborative Filtering Based Recommendation Systems**

### For more accurate results only consider users who have rated atleast 100 books

In [38]:
ratings

,userID,ISBN,bookRating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
8,276744,038550120X,7
16,276747,0060517794,9
19,276747,0671537458,9
20,276747,0679776818,8
21,276747,0943066433,7
23,276747,1885408226,7
24,276748,0747558167,6


In [39]:
g = ratings.groupby('userID')
count_df=g.count()
ratings = ratings.iloc[count_df.loc[count_df.ISBN >100].index]
#ratings[ratings_done['userID'].isin(df_user['userID'])]
ratings

,userID,ISBN,bookRating
9400,278773,0345457374,9
9589,14,0689821166,6
10187,254,0380810492,7
14078,2030,0385484100,10
14709,2148,0316182540,8
17922,3363,0064470679,8
20394,3827,0743448782,8
20407,3827,0876056354,9
20925,4017,0786866470,9
20987,4049,0745321992,9


### Generating ratings matrix from explicit ratings


#### Note: since NaNs cannot be handled by training algorithms, replace these by 0, which indicates absence of ratings

In [44]:
user_bookrating_pivot = ratings.pivot_table(index='userID', columns='ISBN', values='bookRating')
user_bookrating_pivot.head(20)

ISBN,0006176747,0006512208,0060168218,0060172223,0060391626,0060502258,0060505885,0060512806,0060527048,0060920335,...,3548310702,3791313657,8420618578,8420654426,8423918106,8433914952,8433925180,8440688288,8838917299,9060055608
userID,,,,,,,,,,,,,,,,,,,,,
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
#replacing Nan's to 0
user_bookrating_pivot = user_bookrating_pivot.replace(np.nan, 0)
user_bookrating_pivot.head()

ISBN,0006176747,0006512208,0060168218,0060172223,0060391626,0060502258,0060505885,0060512806,0060527048,0060920335,...,3548310702,3791313657,8420618578,8420654426,8423918106,8433914952,8433925180,8440688288,8838917299,9060055608
userID,,,,,,,,,,,,,,,,,,,,,
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
user_bookrating_pivot.shape

(385, 435)

### Generate the predicted ratings using SVD with no.of singular values to be 50

In [58]:

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
svd.fit(user_bookrating_pivot)  

svd.singular_values_


array([26.2297541 , 23.02172887, 17.69180561, 17.32050777, 16.76305292,
       16.18641151, 14.59444813, 14.14208546, 14.14204644, 14.14200547,
       14.07095075, 13.45345547, 13.45323976, 13.45303009, 13.45256841,
       13.42462346, 12.80491785, 12.80398618, 12.72542584, 12.03939865,
       12.03513795, 12.03302469, 12.0163    , 11.37889723, 11.19523107,
       11.10174639, 10.54248873, 10.50366785,  9.99537107,  9.99406471,
        9.99232508,  9.9904867 ,  9.99020418,  9.9885224 ,  9.98670581,
        9.98560934,  9.98459125,  9.98282122,  9.98085812,  9.97849853,
        9.97469144,  9.9715805 ,  9.97027057,  9.96661209,  9.96491491,
        9.95749143,  9.9478847 ,  9.94329649,  9.9378716 ,  9.93326562])

### Take a particular user_id

### Lets find the recommendations for user with id `2110`

#### Note: Execute the below cells to get the variables loaded

In [56]:
userID = 2110

In [441]:
user_id = 2 #2nd row in ratings matrix and predicted matrix

### Get the predicted ratings for userID `2110` and sort them in descending order

In [59]:
 pred_rating = svd.(user_id, user_id)

AttributeError: 'TruncatedSVD' object has no attribute 'predict'

### Create a dataframe with name `user_data` containing userID `2110` explicitly interacted books

In [442]:
user_data.head()

NameError: name 'user_data' is not defined

In [ ]:
user_data.shape

### Combine the user_data and and corresponding book data(`book_data`) in a single dataframe with name `user_full_info`

In [ ]:
book_data.shape

In [ ]:
book_data.head()

In [ ]:
user_full_info.head()

### Get top 10 recommendations for above given userID from the books not already rated by that user